reference:
- (aihub): http://aihub.or.kr/sites/default/files/2019-12/%ED%95%9C%EA%B5%AD%EC%96%B4%20%EC%9D%8C%EC%84%B1%20%EC%A0%84%EC%82%AC%EA%B7%9C%EC%B9%99%20v1.0.pdf
- (audio file understanding): https://m.blog.naver.com/PostView.nhn?blogId=yeominrock&logNo=167235723&proxyReferer=https%3A%2F%2Fwww.google.com%2F

In [1]:
import sys
sys.path.insert(0, '../')

In [70]:
import os
import glob
import numpy as np
import torch
from feature import create_mfcc
from modeling import Listener, Speller, LAS

In [3]:
maxlen = 77
input_size = 39
listener_hidden_size = 256
speller_hidden_size = 2*listener_hidden_size
n_batch = 1
n_char = 63  # 61 classes + sos & eos

In [67]:
61.2

ls: cannot access '../wave/0.and': No such file or directory


In [4]:
inp, inp_shape = create_mfcc('../wave/0.wav')
inp.shape, inp_shape

((704, 39), 704)

In [5]:
# make random input
inp = torch.FloatTensor(inp).unsqueeze(dim=0)
lbl = np.random.randint(0, n_char, (n_batch, maxlen))

In [6]:
inp.shape, lbl.shape

(torch.Size([1, 704, 39]), (1, 77))

In [7]:
model = LAS(
    input_size=input_size,
    listener_hidden_size=listener_hidden_size,
    speller_hidden_size=speller_hidden_size,
    output_class_dim=n_char,
    max_label_len=maxlen,
)

In [8]:
raw_pred_seq, attention_record = model(inp)
print('raw_pred_seq: {}'.format(len(raw_pred_seq)))
print('raw_pred_seq[0]: {}'.format(raw_pred_seq[0].shape))
print('attention_record: {}'.format(len(attention_record)))

raw_pred_seq: 77
raw_pred_seq[0]: torch.Size([1, 63])
attention_record: 77


### input file

In [1]:
import sys
sys.path.insert(0, '../Korean-Speech-Recognition/')

In [2]:
import glob
import queue
import os
from package.dataset import BaseDataset, split_dataset
from package.config import Config
from package.definition import EOS_TOKEN, logger, SOS_TOKEN
from package.loader import load_data_list, load_targets, MultiLoader, BaseDataLoader

FileNotFoundError: [Errno 2] No such file or directory: './data/label/train_labels.csv'

In [ ]:
data_path = '/data_hdd/stt/KsponSpeech_sample'

In [3]:
pcm_files = sorted(glob.glob(os.path.join(data_path, '*.pcm')))
txt_files = sorted(glob.glob(os.path.join(data_path, '*.txt')))

NameError: name 'data_path' is not defined

In [5]:
for t, p in zip(txt_files, pcm_files):
    print(t, p)
    break

/data_hdd/stt/KsponSpeech_sample/KsponSpeech_000001.txt /data_hdd/stt/KsponSpeech_sample/KsponSpeech_000001.pcm


In [6]:
# BaseDataset(
#     audio_paths=pcm_files,
#     label_paths=txt_files,
#     sos_id=SOS_TOKEN, eos_id=EOS_TOKEN,
#     target_dict=target_dict,
#     input_reverse=config.input_reverse,
#     use_augment=config.use_augment,
#     batch_size=config.batch_size,
#     augment_ratio=config.augment_ratio,
#     pack_by_length=config.pack_by_length
# )

In [7]:
config = Config(
    use_bidirectional = True,
    use_attention = True,
    use_label_smooth = True,
    input_reverse = True,
    use_augment = True,
    use_pickle = False,
    use_pyramidal = False,
    use_cuda = True,
    pack_by_length = True,
    augment_ratio = 1.0,
    hidden_size = 256,
    dropout = 0.5,
    listener_layer_size = 5,
    speller_layer_size = 3,
    batch_size = 6,
    worker_num = 1,
    max_epochs = 40,
    use_multistep_lr = False,
    init_lr = 0.0001,
    high_plateau_lr = 0.0003,
    low_plateau_lr = 0.00001,
    teacher_forcing = 0.90,
    seed = 1,
    max_len = 151
)

[2020-04-01 17:58:12,376 config.py:88 - print_log()] use_bidirectional : True
[2020-04-01 17:58:12,377 config.py:89 - print_log()] use_attention : True
[2020-04-01 17:58:12,378 config.py:90 - print_log()] use_pickle : False
[2020-04-01 17:58:12,379 config.py:91 - print_log()] use_augment : True
[2020-04-01 17:58:12,380 config.py:92 - print_log()] use_pyramidal : False
[2020-04-01 17:58:12,381 config.py:93 - print_log()] augment_ratio : 1.00
[2020-04-01 17:58:12,383 config.py:94 - print_log()] input_reverse : True
[2020-04-01 17:58:12,384 config.py:95 - print_log()] hidden_size : 256
[2020-04-01 17:58:12,385 config.py:96 - print_log()] listener_layer_size : 5
[2020-04-01 17:58:12,387 config.py:97 - print_log()] speller_layer_size : 3
[2020-04-01 17:58:12,388 config.py:98 - print_log()] dropout : 0.50
[2020-04-01 17:58:12,389 config.py:99 - print_log()] batch_size : 6
[2020-04-01 17:58:12,390 config.py:100 - print_log()] worker_num : 1
[2020-04-01 17:58:12,390 config.py:101 - print_log()

In [8]:
if config.use_pickle:
    target_dict = load_pickle(TARGET_DICT_PATH, "load all target_dict using pickle complete !!")
else:
    target_dict = load_targets(txt_files)

100%|██████████| 100/100 [00:00<00:00, 13832.09it/s]

[2020-04-01 17:58:12,464 utils.py:150 - save_pickle()] target_dict save complete !!


In [9]:
target_dict

{'KsponSpeech_000001': '아/ 몬 소리야, 그건 또. b/\n',
 'KsponSpeech_000002': '나는 악습은 원래 없어진다+ 없어져야 된다고 생각하긴 했는데 근데 그/ 약간 필요악으로 하나 정도쯤은 있어야 되거든. 물 뜨러 가고.\n',
 'KsponSpeech_000003': 'b/ n/ 그래서 지호랑 계단 n/ 올라와서 b/ 막 위에 운동하는 기구 있대요. b/ 그서 그걸로 운동 할려구요. b/ n/\n',
 'KsponSpeech_000004': '뭐/ 정신과 병원도 그 약 타서 먹어보고, 그 한동안 연락이 안 된 적이 있었단 말이야. 그때가 언제였 언제였더라?\n',
 'KsponSpeech_000005': 'o/ b/ 그게 (0.1프로)/(영 점 일 프로) 가정의 아이들과 가정의 모습이야? b/\n',
 'KsponSpeech_000006': '그/ 친애하는 판사님께라는 법+ 법 관련 드라마 알고 있어?\n',
 'KsponSpeech_000007': 'o/ 그래가지고 진짜 차 사야겠다 아니 뭐/ 차 안 되면 스쿠터라도 타야되겠다 막/ 그런 생각 들더라구 그래서 운전은 하는 게 좋은 거 같애 진짜 b/\n',
 'KsponSpeech_000008': '그래\n',
 'KsponSpeech_000009': 'o/ 나도 몰라. 나 그/ (3G)/(쓰리 쥐)* 하나도 안 봤음. 어.\n',
 'KsponSpeech_000010': '아/ 내일 나 알바하구나.\n',
 'KsponSpeech_000011': 'n/ 아/ 근데 (1시)/(한 시)에 닫는 게 쫌 아쉽긴 한데 거기 진짜 괜찮은데 b/\n',
 'KsponSpeech_000012': '맞아. 시간 안에 풀고 약간 이것부터 풀고 요런 식으로 풀어보렴. 이런 거 b/\n',
 'KsponSpeech_000013': '삼 점대야? 언니가?\n',
 'KsponSpeech_000014': '한+ 한+ 한 시간에 이 만 원? 거의 이 정도로 이 정도란 말이야. b/\n',
 '

In [10]:
total_time_step, train_dataset_list, valid_dataset = split_dataset(
    config = config,
    audio_paths = pcm_files,
    label_paths = txt_files,
    valid_ratio = 0.015,
    target_dict = target_dict,
)

[2020-04-01 17:58:12,850 dataset.py:196 - split_dataset()] split dataset start !!
[2020-04-01 17:58:12,851 dataset.py:68 - augmentation()] Applying Augmentation...
[2020-04-01 17:58:12,852 utils.py:150 - save_pickle()] 
[2020-04-01 17:58:12,853 utils.py:150 - save_pickle()] 
[2020-04-01 17:58:12,853 dataset.py:247 - split_dataset()] split dataset complete !!


In [11]:
train = train_dataset_list[0]

In [12]:
train.shuffle()

In [13]:
train_queue = queue.Queue(config.worker_num << 1)

In [25]:
train_loader = MultiLoader(train_dataset_list, train_queue, config.batch_size, config.worker_num)

In [26]:
train_loader.start()

Exception in thread Thread-5:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "../Korean-Speech-Recognition/package/loader.py", line 82, in run
    feat, label = self.dataset.get_item(self.idx)
  File "../Korean-Speech-Recognition/package/dataset.py", line 53, in get_item
    label = get_label(self.label_paths[idx], sos_id=self.sos_id, eos_id=self.eos_id, target_dict=self.target_dict)
  File "../Korean-Speech-Recognition/package/utils.py", line 79, in get_label
    labels.append(int(token))
ValueError: invalid literal for int() with base 10: '헬스는'



In [ ]:
a = train_queue.get()

In [ ]:
valid = valid_dataset

### audio/text read

In [1]:
import sys
sys.path.insert(0, '../')

In [2]:
import glob
import re
from tqdm import tqdm
from importlib import import_module as import_module
from preprocess import preprocess_text, preprocess_audio

In [3]:
audio_sample = '/data_hdd/stt/KsponSpeech_sample/KsponSpeech_000001.wav'
text_sample = '/data_hdd/stt/KsponSpeech_sample/KsponSpeech_000001.txt'

In [4]:
wav_files = glob.glob('/data_hdd/stt/KsponSpeech_sample/*.wav')
txt_files = glob.glob('/data_hdd/stt/KsponSpeech_sample/*.txt')

In [5]:
for wav, txt in tqdm(zip(wav_files, txt_files), desc='preprocessing'):
    _mfcc, _ = preprocess_audio(wav)
    _txt = preprocess_text(txt)

preprocessing: 100it [00:02, 41.42it/s]


In [6]:
_mfcc.shape

(419, 39)

In [7]:
_txt

'ㄴㅏ∅ㄴㅡㄴ ㅇㅏㄱㅅㅡㅂㅇㅡㄴ ㅇㅝㄴㄹㅐ∅ ㅇㅓㅄㅇㅓ∅ㅈㅣㄴㄷㅏ∅ ㅇㅓㅄㅇㅓ∅ㅈㅕ∅ㅇㅑ∅ ㄷㅚㄴㄷㅏ∅ㄱㅗ∅ ㅅㅐㅇㄱㅏㄱㅎㅏ∅ㄱㅣㄴ ㅎㅐㅆㄴㅡㄴㄷㅔ∅ ㄱㅡㄴㄷㅔ∅ ㄱㅡ∅ ㅇㅑㄱㄱㅏㄴ ㅍㅣㄹㅇㅛ∅ㅇㅏㄱㅇㅡ∅ㄹㅗ∅ ㅎㅏ∅ㄴㅏ∅ ㅈㅓㅇㄷㅗ∅ㅉㅡㅁㅇㅡㄴ ㅇㅣㅆㅇㅓ∅ㅇㅑ∅ ㄷㅚ∅ㄱㅓ∅ㄷㅡㄴ ㅁㅜㄹ ㄸㅡ∅ㄹㅓ∅ ㄱㅏ∅ㄱㅗ∅'

In [8]:
for txt in txt_files:
    with open(txt, 'r', encoding='cp949') as f:
        _txt = preprocess_text(txt)
        print(txt)
        print(f.readline().rstrip())
        print(_txt)
        print('--------')

/data_hdd/stt/KsponSpeech_sample/KsponSpeech_000011.txt
n/ 아/ 근데 (1시)/(한 시)에 닫는 게 쫌 아쉽긴 한데 거기 진짜 괜찮은데 b/
 ㅇㅏ∅ ㄱㅡㄴㄷㅔ∅ ㅅㅣ∅ㅎㅏㄴ ㅅㅣ∅ㅇㅔ∅ ㄷㅏㄷㄴㅡㄴ ㄱㅔ∅ ㅉㅗㅁ ㅇㅏ∅ㅅㅟㅂㄱㅣㄴ ㅎㅏㄴㄷㅔ∅ ㄱㅓ∅ㄱㅣ∅ ㅈㅣㄴㅉㅏ∅ ㄱㅙㄴㅊㅏㄶㅇㅡㄴㄷㅔ∅ 
--------
/data_hdd/stt/KsponSpeech_sample/KsponSpeech_000099.txt
o/ n/ b/ 쌔 차 사니까 아빠가 맨날 자랑해. b/ 등에서 히터 나오고 b/ 엉덩이에서 히터 나오고 막/ b/ 무슨 기능들이 뭐가 이렇게 많다고 설명하는데 하나도 모르겠어 나는. b/ n/
   ㅆㅐ∅ ㅊㅏ∅ ㅅㅏ∅ㄴㅣ∅ㄲㅏ∅ ㅇㅏ∅ㅃㅏ∅ㄱㅏ∅ ㅁㅐㄴㄴㅏㄹ ㅈㅏ∅ㄹㅏㅇㅎㅐ∅  ㄷㅡㅇㅇㅔ∅ㅅㅓ∅ ㅎㅣ∅ㅌㅓ∅ ㄴㅏ∅ㅇㅗ∅ㄱㅗ∅  ㅇㅓㅇㄷㅓㅇㅇㅣ∅ㅇㅔ∅ㅅㅓ∅ ㅎㅣ∅ㅌㅓ∅ ㄴㅏ∅ㅇㅗ∅ㄱㅗ∅ ㅁㅏㄱ  ㅁㅜ∅ㅅㅡㄴ ㄱㅣ∅ㄴㅡㅇㄷㅡㄹㅇㅣ∅ ㅁㅝ∅ㄱㅏ∅ ㅇㅣ∅ㄹㅓㅎㄱㅔ∅ ㅁㅏㄶㄷㅏ∅ㄱㅗ∅ ㅅㅓㄹㅁㅕㅇㅎㅏ∅ㄴㅡㄴㄷㅔ∅ ㅎㅏ∅ㄴㅏ∅ㄷㅗ∅ ㅁㅗ∅ㄹㅡ∅ㄱㅔㅆㅇㅓ∅ ㄴㅏ∅ㄴㅡㄴ  
--------
/data_hdd/stt/KsponSpeech_sample/KsponSpeech_000013.txt
삼 점대야? 언니가?
ㅅㅏㅁ ㅈㅓㅁㄷㅐ∅ㅇㅑ∅ ㅇㅓㄴㄴㅣ∅ㄱㅏ∅
--------
/data_hdd/stt/KsponSpeech_sample/KsponSpeech_000088.txt
o/ 오/ 신기하다. 음 무슨 알바를 그렇게 신중하게
 ㅇㅗ∅ ㅅㅣㄴㄱㅣ∅ㅎㅏ∅ㄷㅏ∅ ㅇㅡㅁ ㅁㅜ∅ㅅㅡㄴ ㅇㅏㄹㅂㅏ∅ㄹㅡㄹ ㄱㅡ∅ㄹㅓㅎㄱㅔ∅ ㅅㅣㄴㅈㅜㅇㅎㅏ∅ㄱㅔ∅
--------
/data_hdd/stt/KsponSpeech_sample/KsponSpeech_000066.txt
응/ 니가 바리깡 가져오라고 재촉해 가지고 바리깡만 챙겼어. l/
ㅇㅡㅇ ㄴㅣ∅ㄱㅏ∅ ㅂㅏ∅ㄹㅣ∅ㄲㅏㅇ ㄱㅏ∅ㅈㅕ∅ㅇㅗ∅ㄹㅏ∅ㄱㅗ∅ ㅈㅐ∅ㅊㅗㄱㅎㅐ∅ ㄱㅏ∅ㅈㅣ∅ㄱㅗ∅ ㅂㅏ∅ㄹㅣ∅ㄲㅏ

In [23]:
txt = '한 (3만원)/(삼만 원)인가 그럴걸 (aaa)/(ddd)'
txt = '한 (3만원)/(삼만 원)인가 그/ b/ h/ 그럴걸 (aaa)/(ddd)'
txt = '한 (3만원)/(삼만 원)인가 그/ b/ h/ 그럴걸 (aaa)/(ddd)333'

In [24]:
import re

In [25]:
re.sub('\(.*?\)', '더', txt)

'한 더/더인가 그/ b/ h/ 그럴걸 더/더333'

In [26]:
re.search('\(.*?\)', txt)

<_sre.SRE_Match object; span=(2, 7), match='(3만원)'>

In [27]:
amb = re.findall('\(.*?\)\/\(.*?\)', txt)

In [28]:
for ai in amb:
    print(ai.split('/'))

['(3만원)', '(삼만 원)']
['(aaa)', '(ddd)']


In [29]:
def remove_noise(txt):
    txt = re.sub('b\/', 'B', txt)
    txt = re.sub('l\/', 'L', txt)
    txt = re.sub('o\/', 'O', txt)
    txt = re.sub('n\/', 'N', txt)
    return txt

In [30]:
remove_noise(txt)

'한 (3만원)/(삼만 원)인가 그/ B h/ 그럴걸 (aaa)/(ddd)333'

In [31]:
def remove_unnecessary(txt):
    txt = re.sub('[가-힣]\/', 'U', txt)
    return txt

In [32]:
remove_unnecessary(txt)

'한 (3만원)/(삼만 원)인가 U b/ h/ 그럴걸 (aaa)/(ddd)333'

In [38]:
re.findall('\(.+?\)', txt)

['(3만원)', '(삼만 원)', '(aaa)', '(ddd)']

In [35]:
def remove_ambigous(txt):
    #txt = re.sub('\/\(.*?\)', '', txt)
    txt = re.sub('\/\(.+?\)', '', txt)
    #txt = re.sub('[\(|\)]', '', txt)
    return txt

In [36]:
remove_ambigous(txt), txt

('한 (3만원)인가 그/ b/ h/ 그럴걸 (aaa)333',
 '한 (3만원)/(삼만 원)인가 그/ b/ h/ 그럴걸 (aaa)/(ddd)333')

In [19]:
remove_ambigous(txt)

'한 3만원인가 b/ h/ 그럴걸 aaa'

In [20]:
def parsing_text(txt):
    txt = remove_noise(txt)
    txt = remove_ambigous(txt)
    return txt

In [21]:
for txt in txt_files:
    with open(txt, 'r', encoding='cp949') as f:
        original = f.readline().rstrip()
        preprocessed = parsing_text(original)
        print(txt)
        print(original)
        print(preprocessed)
        print('-------')

/data_hdd/stt/KsponSpeech_sample/KsponSpeech_000011.txt
n/ 아/ 근데 (1시)/(한 시)에 닫는 게 쫌 아쉽긴 한데 거기 진짜 괜찮은데 b/
N 아/ 근데 1시에 닫는 게 쫌 아쉽긴 한데 거기 진짜 괜찮은데 B
-------
/data_hdd/stt/KsponSpeech_sample/KsponSpeech_000099.txt
o/ n/ b/ 쌔 차 사니까 아빠가 맨날 자랑해. b/ 등에서 히터 나오고 b/ 엉덩이에서 히터 나오고 막/ b/ 무슨 기능들이 뭐가 이렇게 많다고 설명하는데 하나도 모르겠어 나는. b/ n/
O N B 쌔 차 사니까 아빠가 맨날 자랑해. B 등에서 히터 나오고 B 엉덩이에서 히터 나오고 막/ B 무슨 기능들이 뭐가 이렇게 많다고 설명하는데 하나도 모르겠어 나는. B N
-------
/data_hdd/stt/KsponSpeech_sample/KsponSpeech_000013.txt
삼 점대야? 언니가?
삼 점대야? 언니가?
-------
/data_hdd/stt/KsponSpeech_sample/KsponSpeech_000088.txt
o/ 오/ 신기하다. 음 무슨 알바를 그렇게 신중하게
O 오/ 신기하다. 음 무슨 알바를 그렇게 신중하게
-------
/data_hdd/stt/KsponSpeech_sample/KsponSpeech_000066.txt
응/ 니가 바리깡 가져오라고 재촉해 가지고 바리깡만 챙겼어. l/
응/ 니가 바리깡 가져오라고 재촉해 가지고 바리깡만 챙겼어. L
-------
/data_hdd/stt/KsponSpeech_sample/KsponSpeech_000001.txt
아/ 몬 소리야, 그건 또. b/
아/ 몬 소리야, 그건 또. B
-------
/data_hdd/stt/KsponSpeech_sample/KsponSpeech_000004.txt
뭐/ 정신과 병원도 그 약 타서 먹어보고, 그 한동안 연락이 안 된 적이 있었단 말이야. 그때가 언제였 언제였더라?
뭐/ 정